In [12]:
!pip install keras_genomics
!pip install tensorflow

import keras
from keras import backend as K 
from keras.engine import Layer
from keras.engine.base_layer import InputSpec
from keras.callbacks import History

import keras_genomics
from keras_genomics.layers.convolutional import RevCompConv1D

import tensorflow as tf
import numpy as np

import seqdataloader
import keras.layers as kl
from seqdataloader.batchproducers import coordbased
from seqdataloader.batchproducers.coordbased import coordstovals
from seqdataloader.batchproducers.coordbased import coordbatchproducers
from seqdataloader.batchproducers.coordbased import coordbatchtransformers
from seqdataloader.batchproducers.coordbased.core import Coordinates, KerasBatchGenerator, apply_mask

In [14]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [15]:
import model_archs
from BPNetArchs import RcBPNetArch
from BPNetArchs import SiameseBPNetArch
from BPNetArchs import StandardBPNetArch

In [16]:
import RunBPNetArchs
from RunBPNetArchs import get_inputs_and_targets
from RunBPNetArchs import get_specific_generator
from nRunBPNetArchs import save_all

In [22]:
PARAMETERS = {
    'dataset': 'SOX2',
    'input_seq_len': 1346, 
    'c_task_weight': 0,
    'p_task_weight': 1,
    'filters': 64, 
    'n_dil_layers': 6, 
    'conv1_kernel_size': 21, 
    'dil_kernel_size': 3, 
    'outconv_kernel_size': 75, 
    'optimizer': 'Adam',
    'weight_decay': 0.01,
    'lr': 0.001, 
    'size': 100,
    'kernel_initializer': "glorot_uniform",
    'seed': 1234
}

seq_len = 1346
out_pred_len = 1000

In [23]:
inputs_coordstovals, targets_coordstovals = get_inputs_and_targets(dataset = PARAMETERS['dataset'], seq_len = seq_len, out_pred_len = out_pred_len)

In [24]:
early_stopping_callback = keras.callbacks.EarlyStopping(
                              patience=10, restore_best_weights=True)

In [25]:
def run_model(model, model_arch, PARAMETERS): 
    train_batch_generator, val_batch_generator = get_specific_generator(PARAMETERS, 
                                                                         inputs_coordstovals, 
                                                                         targets_coordstovals, 
                                                                         model_arch, 
                                                                         curr_seed) 
    early_stopping_callback = keras.callbacks.EarlyStopping(
                              patience=10, restore_best_weights=True)
    
    model_history = History()
    model.fit_generator(train_batch_generator,
                        epochs = 200, 
                        validation_data=val_batch_generator,
                        callbacks=[early_stopping_callback, model_history])
    model.set_weights(early_stopping_callback.best_weights)
    save_all(PARAMETERS, model, model_arch, model_history)

In [26]:
np.random.seed(1234)
tf.set_random_seed(1234)

rc_model = RcBPNetArch(is_add = True, **PARAMETERS).get_keras_model()
run_model(model = rc_model, model_arch = "RevComp_half", PARAMETERS = PARAMETERS)

1535
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Epoch 1/200
117/117 [==============================] - 121s 1s/step - loss: 4895.3966 - CHIPNexus.SOX2.logcount_loss: 810.0153 - CHIPNexus.SOX2.profile_loss: 4895.3966 - val_loss: 1839.9045 - val_CHIPNexus.SOX2.logcount_loss: 226.1929 - val_CHIPNexus.SOX2.profile_loss: 1839.9045
Epoch 2/200
117/117 [==============================] - 18s 158ms/step - loss: 3092.9023 - CHIPNexus.SOX2.logcount_loss: 197.0744 - CHIPNexus.SOX2.profile_loss: 3092.9023 - val_loss: 1764.1292 - val_CHIPNexus.SOX2.logcount_loss: 174.0917 - val_CHIPNexus.SOX2.profile_loss: 1764.1292
Epoch 3/200
117/117 [==============================] - 20s 167ms/step - loss: 299

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



117/117 [==============================] - 20s 167ms/step - loss: 712.6052 - CHIPNexus.SOX2.logcount_loss: 106.2281 - CHIPNexus.SOX2.profile_loss: 712.6052 - val_loss: 804.0746 - val_CHIPNexus.SOX2.logcount_loss: 97.5274 - val_CHIPNexus.SOX2.profile_loss: 804.0746
Epoch 62/200
117/117 [==============================] - 20s 167ms/step - loss: 713.4962 - CHIPNexus.SOX2.logcount_loss: 101.3106 - CHIPNexus.SOX2.profile_loss: 713.4962 - val_loss: 786.8999 - val_CHIPNexus.SOX2.logcount_loss: 90.7581 - val_CHIPNexus.SOX2.profile_loss: 786.8999
Epoch 63/200
117/117 [==============================] - 20s 167ms/step - loss: 709.8059 - CHIPNexus.SOX2.logcount_loss: 89.7019 - CHIPNexus.SOX2.profile_loss: 709.8059 - val_loss: 795.7399 - val_CHIPNexus.SOX2.logcount_loss: 90.8151 - val_CHIPNexus.SOX2.profile_loss: 795.7399
Epoch 64/200
 53/117 [============>.................] - ETA: 8s - loss: 687.9283 - CHIPNexus.SOX2.logcount_loss: 88.8235 - CHIPNexus.SOX2.profile_loss: 687.92832143
Heads up: coord

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



117/117 [==============================] - 20s 168ms/step - loss: 726.8760 - CHIPNexus.SOX2.logcount_loss: 66.0974 - CHIPNexus.SOX2.profile_loss: 726.8760 - val_loss: 820.2310 - val_CHIPNexus.SOX2.logcount_loss: 69.5517 - val_CHIPNexus.SOX2.profile_loss: 820.2310
Epoch 71/200
117/117 [==============================] - 19s 165ms/step - loss: 763.6203 - CHIPNexus.SOX2.logcount_loss: 64.7974 - CHIPNexus.SOX2.profile_loss: 763.6203 - val_loss: 819.6838 - val_CHIPNexus.SOX2.logcount_loss: 58.8450 - val_CHIPNexus.SOX2.profile_loss: 819.6838
Epoch 72/200
117/117 [==============================] - 20s 167ms/step - loss: 726.4070 - CHIPNexus.SOX2.logcount_loss: 49.8270 - CHIPNexus.SOX2.profile_loss: 726.4070 - val_loss: 829.4321 - val_CHIPNexus.SOX2.logcount_loss: 51.7433 - val_CHIPNexus.SOX2.profile_loss: 829.4321
Epoch 73/200
117/117 [==============================] - 19s 166ms/step - loss: 772.8794 - CHIPNexus.SOX2.logcount_loss: 52.1190 - CHIPNexus.SOX2.profile_loss: 772.8794 - val_loss: 79

...

117/117 [==============================] - 20s 169ms/step - loss: 805.8035 - CHIPNexus.SOX2.logcount_loss: 528.8355 - CHIPNexus.SOX2.profile_loss: 805.8035 - val_loss: 819.2778 - val_CHIPNexus.SOX2.logcount_loss: 562.9201 - val_CHIPNexus.SOX2.profile_loss: 819.2778
Epoch 14/200
117/117 [==============================] - 19s 165ms/step - loss: 794.1452 - CHIPNexus.SOX2.logcount_loss: 563.4971 - CHIPNexus.SOX2.profile_loss: 794.1452 - val_loss: 805.2079 - val_CHIPNexus.SOX2.logcount_loss: 541.8643 - val_CHIPNexus.SOX2.profile_loss: 805.2079
Epoch 15/200
117/117 [==============================] - 20s 167ms/step - loss: 772.6367 - CHIPNexus.SOX2.logcount_loss: 499.4696 - CHIPNexus.SOX2.profile_loss: 772.6367 - val_loss: 796.4610 - val_CHIPNexus.SOX2.logcount_loss: 476.6546 - val_CHIPNexus.SOX2.profile_loss: 796.4610
Epoch 16/200
117/117 [==============================] - 19s 166ms/step - loss: 758.2298 - CHIPNexus.SOX2.logcount_loss: 460.2802 - CHIPNexus.SOX2.profile_loss: 758.2298 - val_l

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



117/117 [==============================] - 19s 165ms/step - loss: 737.0189 - CHIPNexus.SOX2.logcount_loss: 51.8672 - CHIPNexus.SOX2.profile_loss: 737.0189 - val_loss: 768.7721 - val_CHIPNexus.SOX2.logcount_loss: 53.4455 - val_CHIPNexus.SOX2.profile_loss: 768.7721
Epoch 19/200
117/117 [==============================] - 19s 166ms/step - loss: 724.6565 - CHIPNexus.SOX2.logcount_loss: 51.9656 - CHIPNexus.SOX2.profile_loss: 724.6565 - val_loss: 760.3773 - val_CHIPNexus.SOX2.logcount_loss: 53.3291 - val_CHIPNexus.SOX2.profile_loss: 760.3773
Epoch 20/200
117/117 [==============================] - 19s 165ms/step - loss: 705.7748 - CHIPNexus.SOX2.logcount_loss: 51.7719 - CHIPNexus.SOX2.profile_loss: 705.7748 - val_loss: 758.4599 - val_CHIPNexus.SOX2.logcount_loss: 52.2397 - val_CHIPNexus.SOX2.profile_loss: 758.4599
Epoch 21/200
117/117 [==============================] - 19s 164ms/step - loss: 704.4487 - CHIPNexus.SOX2.logcount_loss: 50.7372 - CHIPNexus.SOX2.profile_loss: 704.4487 - val_loss: 75

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



117/117 [==============================] - 19s 164ms/step - loss: 698.3689 - CHIPNexus.SOX2.logcount_loss: 35.8481 - CHIPNexus.SOX2.profile_loss: 698.3689 - val_loss: 746.6444 - val_CHIPNexus.SOX2.logcount_loss: 35.6771 - val_CHIPNexus.SOX2.profile_loss: 746.6444
Epoch 38/200
117/117 [==============================] - 19s 166ms/step - loss: 696.2428 - CHIPNexus.SOX2.logcount_loss: 34.6595 - CHIPNexus.SOX2.profile_loss: 696.2428 - val_loss: 749.2390 - val_CHIPNexus.SOX2.logcount_loss: 35.6915 - val_CHIPNexus.SOX2.profile_loss: 749.2390
Epoch 39/200
117/117 [==============================] - 19s 167ms/step - loss: 699.2301 - CHIPNexus.SOX2.logcount_loss: 35.4765 - CHIPNexus.SOX2.profile_loss: 699.2301 - val_loss: 745.9097 - val_CHIPNexus.SOX2.logcount_loss: 33.5411 - val_CHIPNexus.SOX2.profile_loss: 745.9097
Epoch 40/200
117/117 [==============================] - 19s 165ms/step - loss: 696.5357 - CHIPNexus.SOX2.logcount_loss: 33.9773 - CHIPNexus.SOX2.profile_loss: 696.5357 - val_loss: 75